In [1]:
import tensorflow as tf

x = tf.Variable(3, name = "x")
y = tf.Variable(4, name = "y")
f = x*x*y + y+ 2

with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    

In [2]:
print(result)

42


In [ ]:

import warnings; warnings.simplefilter("ignore")
#importing important libraries
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.formula.api as  sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import csv

df = pd.read_csv("diffreport.csv", sep= ",")

d1 = df.drop("name", axis = 1)
d2 = d1.drop("isotopes", axis = 1)
d3 = d2.drop("adduct", axis = 1)
d4 = d3.drop("tstat", axis = 1)
d5 = d4.drop("pvalue", axis = 1)
d6 = d5.drop("fold", axis = 1)
d7 = d6.drop(d6.columns[0], axis = 1)
d8 = d7.drop("npeaks", axis = 1)
d9 = d8.drop("Eta6", axis = 1)
d10 = d9.drop("Eta8", axis = 1)
columns = ['mzmin', 'mzmax', 'rtmed', 'rtmin', 'rtmax', 'Eta6_0', 'Eta6_2', 'Eta6_3', 'Eta8.1', 'Eta82']
df1 = pd.DataFrame(d10, columns = columns)
df1.rename(columns = {'Eta6_0': 'target'})

In [13]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype = tf.float32, name = "X")
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = "y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session()  as sess:
    theta_value = theta.eval()


In [23]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(housing_data_plus_bias, dtype = tf.float32,  name = "X")
y = tf.constant(housing.target.reshape(-1, 1),dtype = tf.float32, name = "y")
theta = tf.Variable(tf.random_uniform([n +1, 1], -1.0, 1.0), name = "theta")
y_pred = tf.matmul(X, theta, name = "predictions")
error = y_pred -y

mse = tf.reduce_mean(tf.square(error), name = "mse")
gradients = 2/m *tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta-learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session()as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE=", mse.eval())
        sess.run(training_op)
    
    
    best_theta = theta.eval()

Epoch 0 MSE= 28438.5
Epoch 100 MSE= nan
Epoch 200 MSE= nan
Epoch 300 MSE= nan
Epoch 400 MSE= nan
Epoch 500 MSE= nan
Epoch 600 MSE= nan
Epoch 700 MSE= nan
Epoch 800 MSE= nan
Epoch 900 MSE= nan


In [24]:
#Using an optimizer

optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(mse)

# Momentum optimizer is faster
optimizer = tf.train.MomentumOptimizer(learning_rate = learning_rate,momentum = 0.9)

In [25]:
#using mini batch gradient descent
X = tf.placeholder(tf.float32, shape = (None, n +1), name = "X")
y = tf.placeholder(tf.float32,shape = (None, n +1), name = "y") 

batch_size = 100
n_batches = int(np.ceil(m/batch_size))


In [ ]:
#executon phase, fetch the mini batches one by one
theta = tf.Variable(tf.random_uniform([n +1, 1], -1.0, 1.0), name = "theta")
init = tf.global_variables_initializer()
saver = tf.train.Saver()

for batch_index in range (n_batches):
    X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
    if batch_index % 10 == 0:
        summary_str = mse_summary.eval(feed_dict = {X: X_batch, y:y_batch})
        step = epoch * n_batches + batch_index
        file_writer.add_summary(summary_str, step)
        sess.run(training_op, feed_dict = {X: X_batch, y:y_batch})


def fetch_batch(epoch,batch_index, batch_size):
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        X_batch, y_batch= fetch_batch(epoch,batch_index,  batch_size)
        sess.run(training_op, feed_dict = {X: X_batch, y:y_batch})
        save_path = saver.save(sess, "/tmp/my_model.ckpt")  # creating a checkpoint to save model incase it crashes
        
        best_theta = theta.eval()
        save_path = saver.save(sess, "/tmp/my_model_final.ckpt")
        
    

In [ ]:
#restoring a tensor graph and stored variables
saver = tf.train.import_meta_graph("/tmp./my_model_final.ckpt.meta")
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")

## Visualizing the Graph and Training Curves Using TensorBoard

In [ ]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)



# code at the end of the construction phase
mse_summary = tf.summary.scalar('MSE', mse) # creates node in graph and stores MSE value to TensorBoard
file_writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

